In [1]:
VERSION = 2

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from pathlib import Path
import json
import requests
from redis.client import Redis
from redis.commands.search.field import TextField, VectorField
from redis.exceptions import ResponseError
import numpy as np
from redis.commands.search.query import Query
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from textwrap import dedent
from typing import Generator, Tuple
from IPython.display import display, HTML
from embeddings import Embeddings
from embedding_storage.redis_storage import RedisEmbeddingStorage

In [3]:
def display_fact(text):
    display(HTML(f'<div style="background-color: #00ff00;">FACT! <pre>{text}</pre></div>'))

In [4]:
with open(Path().absolute() / '..' / 'dev-config.json', 'r') as f:
    CONFIG = json.load(f)

In [5]:
class Chat:
    MORE = 'MORE_INFO_NEEDED'
    
    def __init__(self, *, config, embeddings):
        self._config = config
        self._embeddings = embeddings
        
        self._chat0 = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=config['openai_api_key'], max_tokens=1200, model_kwargs={'temperature': 0})
        self._chat1 = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=config['openai_api_key'], max_tokens=1200, model_kwargs={'temperature': 1.0})
        self._system_prompt = dedent(
            f"""
            I am a helpful assistant with access to memory.
            Before answering a question, I ABSOLUTELY MUST access my memory by doing the folowing:
                * I write `{self.MORE}`: QUERY, where QUERY is what I feel missing from my knowledge.
            I never use the same QUERY twice.
            Never use the same query twice, always come up with something new.
            Remeber that we can do it more than once to find an answer.
            """.strip() + '\n'
        )
        self.reset()
        
    def _ask_chat(self, lst: list) -> str:
        print('---')
        for x in lst:
            print(f'[{type(x).__name__}]')
            print(x.content)
        print('---')
        print('')
        
        return self._chat0(lst)
        
    def __call__(self, text) -> str:
        return self.say(text)
    
    def _info_requests_message(self, info_requests: list) -> AIMessage:
        if info_requests:
            return AIMessage(content='There are the things I ask previously:\n{}'.format('\n'.join(
                f'  * {self.MORE}: {m}' for m in info_requests
            )))
        else:
            return None
    
    def _facts_message(self, facts: list) -> AIMessage:
        if facts:
            return AIMessage(content='This is what I remembered:\n{}'.format('\n'.join(
                f'  * {f}' for f in facts
            )))
        else:
            return None
    
    def _trim(self, messages):
        available = 3000
        for m in messages:
            if len(m.content) <= available:
                yield m
                available -= len(m.content)
            else:
                m_copy = m.copy()
                m_copy.content = m.content[:available]
                yield m_copy
                available = 0
                
            if not available:
                break
    
    def reset(self) -> None:
        self._history = []
    
    def say(self, text, facts: list = None, info_requests: list = None, depth=0) -> str:
        if facts is None:
            facts = []
        if info_requests is None:
            info_requests = []
        
        to_send = [
            AIMessage(content=self._system_prompt),
            HumanMessage(content=text),
        ]
        for m in (self._info_requests_message(info_requests), self._facts_message(facts)):
            if m is not None:
                to_send.append(m)
        to_send = list(self._trim(to_send))
        response = self._ask_chat(to_send)
        response_text = response.content
        
        if response_text.startswith(self.MORE) and depth < 2:
            request = response_text[len(self.MORE) + 2:]
            print(f'[*] New REQUEST! {request}')
            if request not in info_requests:
                info_requests.append(request)
                new_facts = False
                docs = self._embeddings.knn(request, k=2)
                print(f'DOCS {len(docs)}')
                for text in docs:
                    if text not in facts:
                        facts.append(text)
                        new_facts = True
                        display_fact(text)
                        #print(f'[*] New fact! {text}')
                if new_facts:
                    return self.say(text, depth=depth+1, facts=facts, info_requests=info_requests)
                else:
                    print('[*] No new facts')
            else:
                print('[*] Duplicate request')

        return response


In [6]:
rds = Redis(host='localhost', port=6379, db=0)
storage = RedisEmbeddingStorage('exmaplegpt', redis=rds)
e = Embeddings(config=CONFIG, version=VERSION, storage=storage)
chat = Chat(config=CONFIG, embeddings=e)

In [7]:
chat.reset(); chat('Чем махнул Каратаев в повести Тургенева "Бежин луг"?')

---
[AIMessage]
I am a helpful assistant with access to memory.
            Before answering a question, I ABSOLUTELY MUST access my memory by doing the folowing:
                * I write `MORE_INFO_NEEDED`: QUERY, where QUERY is what I feel missing from my knowledge.
            I never use the same QUERY twice.
            Never use the same query twice, always come up with something new.
            Remeber that we can do it more than once to find an answer.

[HumanMessage]
Чем махнул Каратаев в повести Тургенева "Бежин луг"?
---

[*] New REQUEST! Какое действие совершил Каратаев в повести "Бежин луг"?
DOCS 2


---
[AIMessage]
I am a helpful assistant with access to memory.
            Before answering a question, I ABSOLUTELY MUST access my memory by doing the folowing:
                * I write `MORE_INFO_NEEDED`: QUERY, where QUERY is what I feel missing from my knowledge.
            I never use the same QUERY twice.
            Never use the same query twice, always come up with something new.
            Remeber that we can do it more than once to find an answer.

[HumanMessage]
Part of document called Бежин луг, из цикла "Записки охотника". Starts from char 38700 out of 39742.
---

мое ответило ему легкой, веселой дрожью. Я проворно встал и подошел к мальчикам. Они все спали как убитые вокруг тлеющего костра; один Павел приподнялся до половины и пристально поглядел на меня.
   Я кивнул ему головой и пошел восвояси вдоль задымившейся реки. Не успел я отойти двух верст, как уже полились кругом меня по широкому мокрому лугу, и спереди, по зазеленевшимся холмам, от лесу до лесу, и сзади по

AIMessage(content='I apologize, but I couldn\'t find any information about the specific action that Karataev performs in the story "Бежин луг" from the cycle "Записки охотника". It seems that the information I have access to does not include that specific detail. Is there anything else I can help you with?', additional_kwargs={}, example=False)

In [ ]:
class Indexer:
    PATTERN = "Part of document called {title}. Starts from char {offset} out of {length}.\n---\n\n{chunk}"
    
    def __init__(self, *, embeddings: Embeddings, title: str, text: str):
        self._e = embeddings
        self._title = title
        self._text = text
        
        self._window = 10000
        self._overlap = 1000
        
        assert self._window > self._overlap
        
    def _chunks(self) -> Generator[Tuple[int, str], None, None]:
        offset = 0
        length = len(self._text)
        while True:
            if length - offset < 1.5 * self._window:
                yield (offset, self._text[offset:])
                return
            else:
                yield (offset, self._text[offset : offset + self._window])
                offset += self._window - self._overlap
        
    def index(self) -> None:
        for offset, chunk in self._chunks():
            text_to_index = self.PATTERN.format(title=self._title, offset=offset, length=len(self._text), chunk=chunk)
            self._e.add(text_to_index)

In [ ]:
e = Embeddings('examplegpt', config=CONFIG)
with open('lug.txt') as f:
    text = f.read()
indexer = Indexer(embeddings=e, title='Бежин луг, из цикла "Записки охотника"', text=text)
indexer.index()